In [1]:
import findspark

findspark.init(spark_home=r'D:\ProgramFiles\Spark\spark-3.2.3-bin-hadoop3.2')
findspark.find()

'D:\\ProgramFiles\\Spark\\spark-3.2.3-bin-hadoop3.2'

In [2]:
import pyspark
pyspark.__file__

'D:\\ProgramFiles\\Spark\\spark-3.2.3-bin-hadoop3.2\\python\\pyspark\\__init__.py'

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df_fhvhv = spark.read.option('header','true').parquet('fhvhv_tripdata_2021-06.parquet')

In [5]:
df_zone = spark.read.option('header','true').csv('taxi_zone_lookup.csv')

In [6]:
df_fhvhv.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02682|2021-06-03 21:14:32|2021-06-03 21:25:50|         231|         114|      N|                B02682|
|              B02880|2021-06-23 17:40:44|2021-06-23 18:04:19|          41|         163|      N|                B02880|
|              B02510|2021-06-06 02:07:03|2021-06-06 02:24:30|          75|         250|      N|                  null|
|              B02865|2021-06-06 04:48:19|2021-06-06 05:15:14|         158|          22|      N|                B02865|
|              B02872|2021-06-16 15:31:51|2021-06-16 16:18:09|          90|          56|      N|                B02872|
|              B02883|2021-06-21 20:39:2

In [7]:
df_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [8]:
df_fhvhv.registerTempTable('fhvhv_table')
df_zone.registerTempTable('zone_table')

D:\ProgramFiles\Spark\spark-3.2.3-bin-hadoop3.2\python\pyspark\sql\dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [26]:
spark.sql(
"""
SELECT
    zpu.Zone,
    COUNT(1) AS Zone_Count
FROM
    fhvhv_table AS fhv,
    zone_table AS zpu,
    zone_table AS zdo
WHERE
    fhv.PULocationID =  zpu.LocationID AND
    fhv.DOLocationID = zdo.LocationID
GROUP BY 
    zpu.Zone
ORDER BY Zone_Count DESC
;
"""
).show()

+--------------------+----------+
|                Zone|Zone_Count|
+--------------------+----------+
| Crown Heights North|    231279|
|        East Village|    221244|
|         JFK Airport|    188867|
|      Bushwick South|    187929|
|       East New York|    186780|
|TriBeCa/Civic Center|    164344|
|   LaGuardia Airport|    161596|
|            Union Sq|    158937|
|        West Village|    154698|
|             Astoria|    152493|
|     Lower East Side|    151020|
|        East Chelsea|    147673|
|Central Harlem North|    146402|
|Williamsburg (Nor...|    143683|
|          Park Slope|    143594|
|  Stuyvesant Heights|    141427|
|        Clinton East|    139611|
|West Chelsea/Huds...|    139431|
|             Bedford|    138428|
|         Murray Hill|    137879|
+--------------------+----------+
only showing top 20 rows

